# 6.4 例題: ベイズ多腕バンディット

In [22]:
using Distributions

In [15]:
struct Bandits
    p::Vector{Float64}
end

function pull(b::Bandits, i::T) where {T <: Integer}
    rand() < b.p[i]
end

length(b::Bandits) = Base.length(b.p)

length (generic function with 1 method)

In [16]:
x = Bandits([0.4, 0.2])

Bandits([0.4, 0.2])

In [18]:
pull(x, 2)

false

In [17]:
length(x)

2

In [52]:
mutable struct BayesianStrategy
    bandits::Bandits
    wins::Vector{Float64}
    trials::Vector{Int64}
    N::Int64
    choices::Vector{Float64}
    bb_score::Vector{Bool}
end

function BayesianStrategy(b::Bandits)
    n_bandits = length(b)
    BayesianStrategy(b, zeros(n_bandits), zeros(n_bandits), 0, [], [])
end

function sample_bandits(b::BayesianStrategy, n::Int64)
    bb_score = zeros(n)
    choices = zeros(n)
    for k in 1:n
        # バンディットの事前分布からサンプリングし、最も値の大きいサンプルを選択
        choice = argmax(rand.(Beta.(1 .+ b.wins, 1 .+ b.trials - b.wins)))
        
        # 選択されたバンディットからサンプリング
        result = pull(b.bandits, choice)
        
        b.wins[choice] += result
        b.trials[choice] += 1
        bb_score[k] = result
        b.N += 1
        choices[k] = choice
    end
    
    b.bb_score = [b.bb_score..., bb_score...]
    b.choices = [b.choices..., choices...]
    b
end

sample_bandits (generic function with 1 method)

In [53]:
y = BayesianStrategy(x)

BayesianStrategy(Bandits([0.4, 0.2]), [0.0, 0.0], [0, 0], 0, Float64[], Bool[])

In [56]:
sample_bandits(y, 3)

BayesianStrategy(Bandits([0.4, 0.2]), [3.0, 1.0], [3, 3], 6, [2.0, 1.0, 1.0, 2.0, 2.0, 1.0], Bool[false, true, true, true, false, true])